# Notas de matemática do ENEM 2016

Business Problem: Este [desafio](https://www.codenation.dev/aceleradev/ds-online-1/challenge/enem-ps) faz parte do programa de aceleração da [Codenation](https://www.codenation.dev/). O objetivo é criar um modelo para prever se o canditado é treineiro para dados do ENEM 2016.

Eu utilizei o Google Colab. Assim, fiz o upload dos arquivos no Drive e acessei via PyDrive, conforme as próximas células.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"abc"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_enem4.csv')

In [0]:
downloaded = drive.CreateFile({'id':"abc"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv')

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test_enem4.csv')

In [131]:
print(train.shape, test.shape)

(13730, 167) (4570, 43)


**Datasets**: O conjunto de treino possui dados de 13730 alunos com 167 variáveis para cada um deles. O conjunto de teste possui 43 variáveis e um total de 4570 alunos.

## Análise de dados faltantes (Missing Values)

Os dois conjuntos possuem muitos dados faltantes NaN. Como pode ver observado nas próximas células.

In [0]:
#Função para verificar quais colunas possuem valores nulos
def getMissing(data):
  total = data.isnull().sum().sort_values(ascending=False)
  percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending=False)
  missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
  missing_data.sort_values(by='Total', ascending=False, inplace=True)
  print(missing_data[missing_data['Total'] > 0])

In [133]:
getMissing(train)

                             Total    Percent
CO_UF_ENTIDADE_CERTIFICACAO  12092  88.069920
SG_UF_ENTIDADE_CERTIFICACAO  12092  88.069920
NO_ENTIDADE_CERTIFICACAO     12092  88.069920
Q041                         10792  78.601602
TP_SIT_FUNC_ESC               9448  68.812819
TP_LOCALIZACAO_ESC            9448  68.812819
TP_DEPENDENCIA_ADM_ESC        9448  68.812819
CO_UF_ESC                     9448  68.812819
NO_MUNICIPIO_ESC              9448  68.812819
CO_ESCOLA                     9448  68.812819
TP_ENSINO                     9448  68.812819
SG_UF_ESC                     9448  68.812819
CO_MUNICIPIO_ESC              9448  68.812819
Q033                          7376  53.721777
Q032                          7376  53.721777
Q031                          7376  53.721777
Q028                          7376  53.721777
Q029                          7375  53.714494
Q030                          7375  53.714494
Q027                          7373  53.699927
NU_NOTA_COMP3                 3597

In [134]:
getMissing(test)

                        Total    Percent
TP_DEPENDENCIA_ADM_ESC   3144  68.796499
TP_ENSINO                3144  68.796499
Q027                     2437  53.326039
NU_NOTA_COMP3            1170  25.601751
NU_NOTA_LC               1170  25.601751
NU_NOTA_COMP4            1170  25.601751
NU_NOTA_REDACAO          1170  25.601751
NU_NOTA_COMP2            1170  25.601751
TP_STATUS_REDACAO        1170  25.601751
NU_NOTA_COMP5            1170  25.601751
NU_NOTA_COMP1            1170  25.601751
NU_NOTA_CH               1112  24.332604
NU_NOTA_CN               1112  24.332604


In [0]:
notas = [ 'NU_NOTA_COMP3','NU_NOTA_LC','NU_NOTA_COMP4','NU_NOTA_REDACAO','NU_NOTA_COMP2' ,
         'TP_STATUS_REDACAO','NU_NOTA_COMP5','NU_NOTA_COMP1','NU_NOTA_CH','NU_NOTA_CN']

In [0]:
train[notas] = train[notas].fillna(0)
test[notas] = test[notas].fillna(0)

As colunas dos conjuntos de treino e teste são diferentes. Com isso, vou manter apenas as colunas que existem em ambos os conjuntos.

In [0]:
test_col = test.columns
train_col = train.columns
col_dif = list(set(train_col) - set(test_col))
col_dif.remove('IN_TREINEIRO')

In [0]:
#Remover as colunas do conjunto de treino que não existem no conjunto de teste
train.drop(col_dif, axis=1, inplace=True)

Ainda restaram algumas colunas com valores nulos. Como ultrapassam 50% dos dados, essas colunas serão removidas

In [139]:
getMissing(train)

                        Total    Percent
TP_ENSINO                9448  68.812819
TP_DEPENDENCIA_ADM_ESC   9448  68.812819
Q027                     7373  53.699927


In [140]:
train.groupby("IN_TREINEIRO")['TP_ENSINO'].apply(pd.Series.value_counts)

IN_TREINEIRO     
0             1.0    3886
              3.0     370
              2.0      26
Name: TP_ENSINO, dtype: int64

In [0]:
train["TP_ENSINO"].fillna(4, inplace=True)
test["TP_ENSINO"].fillna(4, inplace=True)

In [0]:
col_drop = ["IN_BAIXA_VISAO",	"IN_CEGUEIRA",	"IN_SURDEZ",	"IN_DISLEXIA",	
            "IN_DISCALCULIA",	"IN_SABATISTA",	"IN_GESTANTE", 'TP_DEPENDENCIA_ADM_ESC', 
            "Q027", "NU_INSCRICAO", "CO_UF_RESIDENCIA", 'TP_NACIONALIDADE',
            'SG_UF_RESIDENCIA']

In [0]:
train.drop(col_drop, axis=1, inplace=True)
test.drop(col_drop, axis=1, inplace=True)

In [144]:
getMissing(test)

Empty DataFrame
Columns: [Total, Percent]
Index: []


In [145]:
print(train.shape, test.shape)

(13730, 31) (4570, 30)


In [146]:
test['TP_ENSINO'].value_counts()

4.0    3144
1.0    1303
3.0     114
2.0       9
Name: TP_ENSINO, dtype: int64

### Análise exploratória dos dados

A partir de agora nossos datasets não possuem mais dados nulos e posso começar a verificar as ligações entre os dados e o que mais impacta na variável target. 

In [147]:
train.head()

,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,IN_IDOSO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q047
0,24,M,1,1,4,1,4.0,0,0,1,1,1,1,436.3,495.4,581.2,1,1.0,120.0,120.0,120.0,80.0,80.0,520.0,D,D,C,A,A,C,A
1,17,F,3,2,0,2,1.0,0,0,1,1,1,1,474.5,544.1,599.0,1,1.0,140.0,120.0,120.0,120.0,80.0,580.0,A,A,B,A,A,A,A
2,21,F,3,3,0,1,4.0,0,0,0,0,0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D,D,C,A,A,A,A
3,25,F,0,1,9,1,4.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,H,E,E,C,B,C,D
4,28,M,2,1,4,1,4.0,0,0,0,0,0,0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,E,D,C,A,A,B,A


In [148]:
train.groupby("IN_TREINEIRO")['TP_ENSINO'].apply(pd.Series.value_counts)

IN_TREINEIRO     
0             4.0    7665
              1.0    3886
              3.0     370
              2.0      26
1             4.0    1783
Name: TP_ENSINO, dtype: int64

In [149]:
train.groupby("IN_TREINEIRO")['IN_IDOSO'].apply(pd.Series.value_counts)

IN_TREINEIRO   
0             0    11945
              1        2
1             0     1783
Name: IN_IDOSO, dtype: int64

In [0]:
from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures
)

In [151]:
#train['TP_IDADE_AGRUPADA'] = pd.cut(train['NU_IDADE'], bins=[0, 17, 19, 21, 26, 31, 41, 100], labels=range(7)).ravel()
discretizer = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")
discretizer.fit(train[['NU_IDADE']])

KBinsDiscretizer(encode='ordinal', n_bins=10, strategy='quantile')

In [0]:
idade_bins = discretizer.transform(train[['NU_IDADE']])
idade_bins_test = discretizer.transform(test[['NU_IDADE']])

In [0]:
train['TP_IDADE_BINS'] = idade_bins
test['TP_IDADE_BINS'] = idade_bins_test

train.drop(["NU_IDADE"], axis=1, inplace=True)
test.drop(["NU_IDADE"], axis=1, inplace=True)

In [0]:
#train.groupby("IN_TREINEIRO")['TP_IDADE_BINS'].apply(pd.Series.value_counts)

In [0]:
col_num = ['NU_NOTA_CN',	'NU_NOTA_CH',	'NU_NOTA_LC',	'NU_NOTA_COMP1',	'NU_NOTA_COMP2',
           'NU_NOTA_COMP3',	'NU_NOTA_COMP4',	'NU_NOTA_COMP5',	'NU_NOTA_REDACAO',
           'Q001',	'Q002',	'Q006',	'Q024',	'Q025',	'Q026',	'Q047']

col_dum = ['TP_SEXO', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
       'TP_ESCOLA', 'TP_ENSINO', 'IN_IDOSO', 'TP_PRESENCA_CN',
       'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'TP_IDADE_BINS']

In [0]:
y_train = train['IN_TREINEIRO']
X_train = train[col_dum]
X_test = test[col_dum]

In [161]:
print(X_train.shape, X_test.shape)

(13730, 14) (4570, 14)


Concatenando os dois conjuntos para aplicar a transformação Dummies das variáveis categóricas e Standard Scaler nas variáveis contínuas.

In [162]:
data = pd.concat([X_train, X_test], axis=0, ignore_index=True)
data.shape

(18300, 14)

In [0]:
data = pd.get_dummies(data, columns=col_dum, drop_first=True)

In [164]:
data.shape

(18300, 51)

In [0]:
#Separando novamente os conjuntos
X_train = data.head(X_train.shape[0])

X_test = data.tail(X_test.shape[0])

In [0]:
from sklearn.metrics import classification_report, accuracy_score 
from xgboost import XGBClassifier

## Modelagem

Utilizei um modelo de regressão XGBClassifier com parâmetros normais. Isso já é suficiente para um bom desempenho. 

In [0]:
xgb_model = XGBClassifier()

xgb_model.fit(X_train, y_train, verbose=False)
y_pred_xgb = xgb_model.predict(X_train)

In [172]:
print(accuracy_score(y_train, y_pred_xgb))

0.9970138383102695


In [173]:
print(classification_report(y_train, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11947
           1       1.00      0.98      0.99      1783

    accuracy                           1.00     13730
   macro avg       1.00      0.99      0.99     13730
weighted avg       1.00      1.00      1.00     13730



O relatório está mostrando um modelo praticamente impecável com dados de treino. Isso pode ser overfitting, vamos ver como ficaram com os dados de teste. 

In [0]:
predicao = xgb_model.predict(X_test)

In [0]:
#Criando um dataframe para submissão
subm = pd.read_csv('test_enem4.csv')

#Fazendo as previsões para o conjunto de test
subm['IN_TREINEIRO'] = predicao

In [0]:
#Separando apenas o número de inscrição e a nota para submeter o desafio
subm = subm[['NU_INSCRICAO','IN_TREINEIRO']]

In [180]:
subm.head()

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0


In [0]:
subm.to_csv("submission_colab.csv", index=False)

In [0]:
#Fazendo download do arquivo de resposta
import os
from google.colab import files

os.chdir(r'/content')

files.download('submission_colab.csv')

### Resultados Submissão

Após submeter o conjunto de teste a acurácia do modelo foi de 99.74%, isso demonstra que apesar de simples o modelo está praticamente perfeito e resolvendo o problema. 